# VDP Functional Tests

#### Version 1.9

Last updated: 04/26/18

In [1]:
import sys
sys.path.append('../')
import vdp
import psycopg2
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

### Single image complete run-through

In [2]:
cfgfile = '/home/erichards/work/vdp/tests/test_config.yaml'
with open(cfgfile, 'r') as f:
    cfg = f.read()
    
print cfg

# vdp configuration file

stages:
                              # processing steps; yes | no  or  True | False
  source finding: yes         # extract sources from image?
  source association: yes     # match extracted sources to existing catalog?
  catalog matching: yes       # cross-match with sky survey catalogs?

options:
                              # processing options; yes | no or True | False
  save to database: yes       # save results to database?
  quality checks: yes         # run image & source count quality checks?
  overwrite: yes              # overwrite database if exists?
  reprocess: no               # redo selected stages even if done previously?
  redo match: no              # redo cross-matching with all sky catalogs?
  update match: no            # update cross-matching with new sky catalogs?

setup:
  root directory: /home/erichards/work/data/test/
  year: 2017
  month: 08
  day: [30]                   # list of days to process; [] = all
  # list files to proce

In [3]:
%run ../vdp.py ../tests/test_config.yaml


###################################################
Starting the VLITE Database Pipeline.
Log file: /home/erichards/work/data/test/201708.log
###################################################
Connected to database functest.
Overwriting existing database tables.


Dropping tables if they exist...
Creating new tables...
_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Adding new entry to image table.
**********************
STAGE 2: SOURCE FINDNG
**********************
Extracting sources...
 -- found 20 sources in 19.01 seconds
Performing source count quality checks...
...image passed.
Adding detected sources to database.
Correcting all flux measurements for primary beam response.


--> Wrote ds9 region file '/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.crop.pybdsm.srl.reg'


***************************
STAGE 3: SOURCE ASSOCIATION
***************************
Extracted 0 sources from assoc_source table within 1.5 degrees.
 -- number of matches: 0
 -- number of new sources to add: 20
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['lofar_lba', 'nrl_nvss', 'nvss', 'sumss', 'wenss']
Attempting to match 20 sources from this image to the lofar_lba sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 0
Attempting to match 20 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the sumss sky catalog...
 -- found previous matching results for 0 sources
 -- 

##### Tests

In [4]:
conn = psycopg2.connect(host='localhost', database='functest', user='erichards')
cur = conn.cursor()

Quick check to make sure **image**, **detected_source**, **corrected_flux**, and **assoc_source** tables are being filled. The number of rows should be the same in all the source tables which should equal 'nsrc' recorded in the **image** table:

In [5]:
cur.execute('SELECT nsrc FROM image')
nsrc = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM detected_source')
dscnt = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM corrected_flux')
cfcnt = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM assoc_source')
ascnt = cur.fetchone()[0]

In [6]:
cntlist = [nsrc, dscnt, cfcnt, ascnt]
equal_count = cntlist.count(cntlist[0]) == len(cntlist)
if not equal_count:
    print('FAILED: table source counts')
else:
    print('OK')

OK


Check various catalog matching results:

In [7]:
# Correct catalogs used
should_use = ['nrl_nvss', 'nvss', 'sumss', 'wenss', 'lofar_lba']
cur.execute('SELECT catalogs_checked FROM image')
did_use = cur.fetchone()[0]

In [8]:
correct_catalogs = set(should_use) == set(did_use)
if not correct_catalogs:
    print('FAILED: wrong catalogs used')
else:
    print('OK')

OK


In [9]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [10]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [11]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [12]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [13]:
cur.close()
conn.close()

___

### Reprocess failure triggers

In [14]:
stages, opts, setup, sfparams, qaparams, dirs = vdp.cfgparse(cfgfile)

In [15]:
opts

{'overwrite': True,
 'quality checks': True,
 'redo match': False,
 'reprocess': False,
 'save to database': True,
 'update match': False}

In [16]:
opts['overwrite'] = False
opts

{'overwrite': False,
 'quality checks': True,
 'redo match': False,
 'reprocess': False,
 'save to database': True,
 'update match': False}

In [17]:
conn = vdp.dbinit(setup['database name'], setup['database user'], opts['overwrite'], qaparams)

Connected to database functest.
Using existing database functest.


In [18]:
# Nothing happens since image is already processed
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Image already processed. Moving on...


In [19]:
opts['reprocess'] = True
stages['source finding'] = False

In [20]:
# Nothing happens since reprocess only works when source finding is turned on
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.

NOTE: /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits's
sources have already been associated with the existing VLITE catalog.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
All specified catalogs with appropriate resolution have already been checked for matches.
Completed sky catalog cross-matching for image
/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.


___

### Successful reprocess

In [21]:
opts

{'overwrite': False,
 'quality checks': True,
 'redo match': False,
 'reprocess': True,
 'save to database': True,
 'update match': False}

In [22]:
stages['source finding'] = True

In [23]:
# Reprocess the image through all stages in the same way as before
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Updating existing entries in image table.
Removing previous sources...
**********************
STAGE 2: SOURCE FINDNG
**********************
Extracting sources...
 -- found 20 sources in 18.55 seconds
Performing source count quality checks...
...image passed.
Adding detected sources to database.
Correcting all flux measurements for primary beam response.


--> Wrote ds9 region file '/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.crop.pybdsm.srl.reg'


***************************
STAGE 3: SOURCE ASSOCIATION
***************************
Extracted 0 sources from assoc_source table within 1.5 degrees.
 -- number of matches: 0
 -- number of new sources to add: 20
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['lofar_lba', 'nrl_nvss', 'nvss', 'sumss', 'wenss']
Attempting to match 20 sources from this image to the lofar_lba sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 0
Attempting to match 20 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the sumss sky catalog...
 -- found previous matching results for 0 sources
 -- 

___

### Redo match

In [24]:
opts['reprocess'] = False
opts['redo match'] = True
opts

{'overwrite': False,
 'quality checks': True,
 'redo match': True,
 'reprocess': False,
 'save to database': True,
 'update match': False}

In [25]:
stages['source finding'] = False
stages['source association'] = False
stages

{'catalog matching': True,
 'source association': False,
 'source finding': False}

In [26]:
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.
Removing previous sky catalog matching results for 20 sources.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['lofar_lba', 'nrl_nvss', 'nvss', 'sumss', 'wenss']
Attempting to match 20 sources from this image to the lofar_lba sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 0
Attempting to match 20 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the nvss sky catalog...
 -- found previous matching

##### Tests

In [27]:
cur = conn.cursor()

In [28]:
# Correct catalogs used
should_use = ['nrl_nvss', 'nvss', 'sumss', 'wenss', 'lofar_lba']
cur.execute('SELECT catalogs_checked FROM image')
did_use = cur.fetchone()[0]

In [29]:
correct_catalogs = set(should_use) == set(did_use)
if not correct_catalogs:
    print('FAILED: wrong catalogs used')
else:
    print('OK')

OK


In [30]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [31]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [32]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [33]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [34]:
cur.close()
conn.close()

___

### Remove catalog matching results

In [35]:
# Remove NVSS, NRL_NVSS, & WENSS
%run ../vdp.py ../tests/test_config.yaml -v 0 --remove_catalog_matches


###################################################
Starting the VLITE Database Pipeline.
Log file: /home/erichards/work/data/test/201708.log
###################################################
Connected to database functest.
Using existing database functest.



For which catalogs would you like to remove matching results? (List catalogs separated by a comma.)
nvss, nrl_nvss, wenss


Removing matching results for nvss, nrl_nvss, wenss...


##### Tests

In [36]:
conn = psycopg2.connect(host='localhost', database='functest', user='erichards')
cur = conn.cursor()

In [37]:
# Correct catalogs used
should_use = ['sumss', 'lofar_lba']
cur.execute('SELECT catalogs_checked FROM image')
did_use = cur.fetchone()[0]

In [38]:
correct_catalogs = set(should_use) == set(did_use)
if not correct_catalogs:
    print('FAILED: wrong catalogs used')
else:
    print('OK')

OK


In [39]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [40]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [41]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [42]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [43]:
cur.close()
conn.close()

___

### Update match - one catalog at a time

In [44]:
stages

{'catalog matching': True,
 'source association': False,
 'source finding': False}

In [45]:
opts['redo match'] = False
opts['update match'] = True
opts

{'overwrite': False,
 'quality checks': True,
 'redo match': False,
 'reprocess': False,
 'save to database': True,
 'update match': True}

In [46]:
setup['catalogs'] = ['nrl_nvss']

In [47]:
conn = vdp.dbinit(setup['database name'], setup['database user'], opts['overwrite'], qaparams)

Connected to database functest.
Using existing database functest.


In [48]:
# Add catalog matching results for NRL_NVSS
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['nrl_nvss']
Attempting to match 20 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Completed sky catalog cross-matching for image
/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.


In [49]:
# Add catalog matching results for NVSS
setup['catalogs'] = ['nvss']
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['nvss']
Attempting to match 20 sources from this image to the nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Completed sky catalog cross-matching for image
/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.


In [50]:
# Add catalog matching results for WENSS
setup['catalogs'] = ['wenss']
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['wenss']
Attempting to match 20 sources from this image to the wenss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Completed sky catalog cross-matching for image
/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.


##### Tests

In [51]:
cur = conn.cursor()

In [52]:
# Correct catalogs used
should_use = ['sumss', 'lofar_lba', 'nrl_nvss', 'nvss', 'wenss']
cur.execute('SELECT catalogs_checked FROM image')
did_use = cur.fetchone()[0]

In [53]:
correct_catalogs = set(should_use) == set(did_use)
if not correct_catalogs:
    print('FAILED: wrong catalogs used')
else:
    print('OK')

OK


In [54]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [55]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [56]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [57]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [58]:
cur.close()

### Update match - add all 3 at once

In [59]:
# Remove them again
%run ../vdp.py ../tests/test_config.yaml -v 0 --remove_catalog_matches


###################################################
Starting the VLITE Database Pipeline.
Log file: /home/erichards/work/data/test/201708.log
###################################################
Connected to database functest.
Using existing database functest.



For which catalogs would you like to remove matching results? (List catalogs separated by a comma.)
nvss, nrl_nvss, wenss


Removing matching results for nvss, nrl_nvss, wenss...


In [60]:
stages, opts, setup, sfparams, qaparams, dirs = vdp.cfgparse(cfgfile)

In [61]:
stages['source finding'] = False
stages['source association'] = False
stages

{'catalog matching': True,
 'source association': False,
 'source finding': False}

In [62]:
opts['overwrite'] = False
opts['update match'] = True
opts

{'overwrite': False,
 'quality checks': True,
 'redo match': False,
 'reprocess': False,
 'save to database': True,
 'update match': True}

In [63]:
setup['catalogs']

['cosmos',
 'first',
 'gleam',
 'gpsr1',
 'gpsr5',
 'lazio04',
 'lofar_hba',
 'lofar_lba',
 'lotss',
 'm31_glg04',
 'nordgc',
 'nrl_nvss',
 'nvss',
 'sevenc',
 'sumss',
 'tgss',
 'txs',
 'vlssr',
 'wenss']

In [64]:
# Now add them all back
process(conn, stages, opts, dirs, setup['files'], setup['catalogs'], sfparams, qaparams)

_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Initializing image.
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['nrl_nvss', 'nvss', 'wenss']
Attempting to match 20 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the nvss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Attempting to match 20 sources from this image to the wenss sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 19
Completed sky catalog cross-matching fo

##### Tests

In [65]:
cur = conn.cursor()

In [66]:
# Correct catalogs used
should_use = ['sumss', 'lofar_lba', 'nrl_nvss', 'nvss', 'wenss']
cur.execute('SELECT catalogs_checked FROM image')
did_use = cur.fetchone()[0]

In [67]:
correct_catalogs = set(should_use) == set(did_use)
if not correct_catalogs:
    print('FAILED: wrong catalogs used')
else:
    print('OK')

OK


In [68]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [69]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [70]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [71]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [72]:
cur.close()
conn.close()

___

### Add another image

In [73]:
with open(cfgfile, 'r') as f:
    cfg = f.read()
    
print cfg

# vdp configuration file

stages:
                              # processing steps; yes | no  or  True | False
  source finding: yes         # extract sources from image?
  source association: yes     # match extracted sources to existing catalog?
  catalog matching: yes       # cross-match with sky survey catalogs?

options:
                              # processing options; yes | no or True | False
  save to database: yes       # save results to database?
  quality checks: yes         # run image & source count quality checks?
  overwrite: no               # overwrite database if exists?
  reprocess: no               # redo selected stages even if done previously?
  redo match: no              # redo cross-matching with all sky catalogs?
  update match: no            # update cross-matching with new sky catalogs?

setup:
  root directory: /home/erichards/work/data/test/
  year: 2017
  month: 08
  day: [30]                   # list of days to process; [] = all
  # list files to proce

In [74]:
%run ../vdp.py ../tests/test_config.yaml -v 0


###################################################
Starting the VLITE Database Pipeline.
Log file: /home/erichards/work/data/test/201708.log
###################################################
Connected to database functest.
Using existing database functest.
_____________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Image already processed. Moving on...
___________________________________________________________________________________
Starting /home/erichards/work/data/test/2017-08/30/Images/6GHz.0137+331.IPln1.fits.
**********************
STAGE 1: READING IMAGE
**********************
Performing preliminary image quality checks...
...image passed.
Adding new entry to image table.
**********************
STAGE 2: SOURCE FINDNG
****************

--> Wrote ds9 region file '/home/erichards/work/data/test/2017-08/30/Images/6GHz.0137+331.IPln1.crop.pybdsm.srl.reg'


...image passed.
Adding detected sources to database.
Correcting all flux measurements for primary beam response.
***************************
STAGE 3: SOURCE ASSOCIATION
***************************
Extracted 20 sources from assoc_source table within 1.5 degrees.
Limiting to sources in resolution class C (35.0" < BMIN <= 60.0")
 -- 20 sources remaining
Attempting to match 45 sources from this image to 20 sources previously detected in VLITE images...
 -- number of matches: 19
 -- number of new sources to add: 26
*********************************
STAGE 4: MATCHING TO SKY CATALOGS
*********************************
Using the following catalogs for cross-matching: ['lofar_lba', 'nrl_nvss', 'nvss', 'sumss', 'wenss']
Attempting to match 26 sources from this image to the lofar_lba sky catalog...
 -- found previous matching results for 0 sources
 -- number of matches: 0
Attempting to match 26 sources from this image to the nrl_nvss sky catalog...
 -- found previous matching results for 0 source

##### Tests

In [75]:
conn = psycopg2.connect(host='localhost', database='functest', user='erichards')
cur = conn.cursor()

Make sure new sources are being added and the number of detections is being incrememted correctly in the **assoc_source** table:

In [76]:
cur.execute('SELECT nsrc FROM image ORDER BY id')
nsrcs = cur.fetchall()
im1nsrc, im2nsrc = nsrcs[0][0], nsrcs[1][0]
cur.execute('SELECT COUNT(1) FROM assoc_source')
ascnt = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM assoc_source WHERE ndetect > 1')
multi_detect_cnt = cur.fetchone()[0]

In [77]:
im1_only = (ascnt - im2nsrc) == (im1nsrc - multi_detect_cnt)
if not im1_only:
    print('FAILED: assoc_source table incorrect after adding new image')
else:
    print('OK')

OK


Check various catalog matching results:

In [78]:
# Sources with no catalog matches inserted into vlite_unique table
cur.execute('SELECT id FROM assoc_source WHERE nmatches = 0')
as_matchless_ids = cur.fetchall()
cur.execute('SELECT assoc_id FROM vlite_unique WHERE detected')
vu_detected_ids = cur.fetchall()

In [79]:
correct_vu_ids = set(as_matchless_ids) == set(vu_detected_ids)
if not correct_vu_ids:
    print('FAILED: incorrect sources in vlite_unique table')
else:
    print('OK')

OK


In [80]:
# Every catalog match recorded in 'nmatches' & catalog_match table
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
sum_nmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [81]:
correct_nmatches = sum_nmatches == cmcnt
if not correct_nmatches:
    print('FAILED: number of matches different than number of sources in catalog_match table')
else:
    print('OK')

OK


In [82]:
# Every VU source detected in one image should have recorded non-detections in the other if there are no
# multi-detection VU sources (ndetect > 1, nmatches = 0)
cur.execute('SELECT COUNT(1) FROM assoc_source WHERE nmatches = 0 AND ndetect > 1')
multi_detect_VU = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM vlite_unique WHERE image_id = 1 AND detected')
im1VU_detect = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM vlite_unique WHERE image_id = 2 AND NOT detected')
im2VU_nodetect = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM vlite_unique WHERE image_id = 2 AND detected')
im2VU_detect = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM vlite_unique WHERE image_id = 1 AND NOT detected')
im1VU_nodetect = cur.fetchone()[0]

In [83]:
same1 = (im1VU_detect - multi_detect_VU) == im2VU_nodetect
same2 = (im2VU_detect - multi_detect_VU) == im1VU_nodetect
if not same1 or not same2:
    print('FAILED: vlite_unique table incorrect')
else:
    print('OK')

OK


In [84]:
cur.close()
conn.close()

___

### Remove an image

In [85]:
%run ../vdp.py ../tests/test_config.yaml -v 0 --remove_image


###################################################
Starting the VLITE Database Pipeline.
Log file: /home/erichards/work/data/test/201708.log
###################################################
Connected to database functest.
Using existing database functest.



Please enter the image(s) filename(s) starting at least with the year-month directory (i.e. 2018-01/15/Images/10GHz.Mrk110.IPln1.fits), or provide a text file with one filename per line:
/home/erichards/work/data/test/2017-08/30/Images/1.5GHz.0137+331.IPln1.fits


Preparing to remove image(s) ['2017-08/30/Images/1.5GHz.0137+331.IPln1.fits'] from the database.



Are you sure? yes


Deleting image(s) from the database...


##### Tests

In [86]:
conn = psycopg2.connect(host='localhost', database='functest', user='erichards')
cur = conn.cursor()

In [87]:
# Make sure the image was deleted from the image table
cur.execute('SELECT COUNT(1) FROM image WHERE id = 1')
if not cur.fetchone()[0] == 0:
    print('FAILED: image still in image table')
else:
    print('OK')

OK


In [88]:
# Make sure sources where removed from detected_source, detected_island, & corrected_flux tables
cur.execute('SELECT COUNT(1) FROM detected_island WHERE image_id = 1')
dicnt = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM detected_source WHERE image_id = 1')
dscnt = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM corrected_flux WHERE image_id = 1')
cfcnt = cur.fetchone()[0]

In [89]:
if not dicnt == 0 or not dscnt == 0 or not cfcnt == 0:
    print('FAILED: sources still in tables')
else:
    print('OK')

OK


In [90]:
# Make sure the only sources in assoc_source are in image 2
cur.execute('SELECT assoc_id FROM detected_source')
ds_assoc_ids = cur.fetchall()
cur.execute('SELECT id FROM assoc_source')
as_ids = cur.fetchall()

In [91]:
same_ids = set(ds_assoc_ids) == set(as_ids)
if not same_ids:
    print('FAILED: incorrect sources in assoc_source')
else:
    print('OK')

OK


In [92]:
# Make sure 'ndetect' was decremented
cur.execute('SELECT COUNT(1) FROM assoc_source WHERE ndetect > 1')
if not cur.fetchone()[0] == 0:
    print('FAILED: wrong ndetect in assoc_source table')
else:
    print('OK')

OK


In [93]:
# Make sure sources were removed from vlite_unique
cur.execute('SELECT COUNT(1) FROM vlite_unique WHERE image_id = 1')
if not cur.fetchone()[0] == 0:
    print('FAILED: sources still in vlite_unique table')
else:
    print('OK')

OK


In [94]:
# Make sure entries were removed from catalog_match
cur.execute('SELECT SUM(nmatches) FROM assoc_source')
asnmatches = cur.fetchone()[0]
cur.execute('SELECT COUNT(1) FROM catalog_match')
cmcnt = cur.fetchone()[0]

In [95]:
if not asnmatches == cmcnt:
    print('FAILED: sources still in catalog_match')
else:
    print('OK')

OK


In [96]:
cur.close()
conn.close()